<a href="https://colab.research.google.com/github/sfiddes/teaching/blob/main/2021_08_ML_workshop/Clustering101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intro**



This notebook steps through k-means and hierarchical clustering in python using the Palmer Penguins data set and SciKit-Learn. 

It has been set up to run via Google Colaboratory, so no installation on your local computer is required. Unfortunately, to take advantage of this, you will need a google account. If you do have a google account: 
- Click Open in Colab at top of notebook (if starting from GitHub)
- Sign into google
- Start running 
- You can Copy to Drive if you want to save any changes you make

Alternatively, you can download the notebook and run it locally, if you have the required packages installed. 
- Click Open in Colab at top of notebook (if starting from GitHub)
- File > Download as .ipynb
- Make sure you have the right packages installed 
- Open up jupyter notebooks and start running 

This notebook assumes little knowledge of python. Most of the code has been hidden to ease readability, but if you want to see what the code is doing, simply hit the 'show code' buttons!  


# **Set-up**
- Install some non-default packages - the dataset Palmer Penguins. Google Colab uses pip (a package installer for python) to do this. 

- Import the packages that we want to use, including:
  - Basic python packages for math/science (numpy, scipy), data handling (pandas), plotting (matplotlib, seaborn)
  - Scikit-Learn - the home of lots of great machine learning packages. We are interested in the clustering packages, as well as some data pre-processing and metric packages
  - Our dataset - Palmer Penguins

In [ ]:
pip install palmerpenguins

In [ ]:
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import preprocessing, metrics

from palmerpenguins import load_penguins

#**Data**
**Import the data and take a look at it**
- Load the data as a Pandas DataFrame
- Try plotting some of the data 

**Questions** 
- What type of data is available (categorical and/or quantitative)?
- What will you try to predict? 
- What will you use to predict it? 
- What shape is the data set distribution? 

In [ ]:
# Import our dataframe - we call it penguins
penguins = load_penguins()

In [ ]:
# Print dataframe (can also use dataframe.head() or dataframe.tail(), or dataframe.variable to isolate columns)
penguins

**Plot a scatter plot (using seaborn)**
- Try to plot a few different variables (eg. predictors) and categories (what we want to predict)
- Can you find a good combination? 

In [ ]:
sns.scatterplot(data=penguins,x='flipper_length_mm',y='body_mass_g',hue='sex',palette='flare');  # change the x, y and hue variables

In [ ]:
#@markdown **Plot a 3D scatter plot** { display-mode: "both" }
c = penguins.species.copy() # need to make categorial groups quantitative [0,1,2]
c[np.where(c=='Adelie')[0]] = 0
c[np.where(c=='Gentoo')[0]] = 1
c[np.where(c=='Chinstrap')[0]] = 2

fig = plt.figure(figsize=(8,5)) # initialise figure
ax = plt.axes(projection='3d')
p = ax.scatter3D(penguins.flipper_length_mm,    # plot in 3D: x
                 penguins.bill_length_mm,       # y
                 penguins.bill_depth_mm,        # z
                 c=c,                           # c (colour)
                 cmap=sns.color_palette('flare',as_cmap=True)); 
cbar = plt.colorbar(p,drawedges=True,ticks=[0,1,2],      # add the colour bar 
                    boundaries=np.arange(-0.5,3.5,1)); 
cbar.set_ticklabels(['Adelie','Gentoo','Chinstrap']) # set color bar tick labels (so not 0,1,2)
ax.view_init(20, 10) # make the viewing angle more optimal
ax.set_xlabel('Flipper Length (mm)');
ax.set_ylabel('Bill Length (mm)');
ax.set_zlabel('Bill Depth (mm)');
plt.title('Observed penguin data');


**How is the data distributed?**

See how the data you want to use as predictors are distributed. This is important as k-means makes the assumption that data is normally distrubuted. 

In [ ]:
# Try your own distribution plot here. Hint: 
#sns.kdeplot(data=dataframe, x=var1, hue=var2) 

In [ ]:
#@markdown Distribution plot example { display-mode: "both" }
fig, axes = plt.subplots(1,4,figsize=(14,4))
sns.kdeplot(data=penguins, x="flipper_length_mm", hue="species", ax=axes[0]); 
sns.kdeplot(data=penguins, x="bill_length_mm", hue="species", ax=axes[1]); 
sns.kdeplot(data=penguins, x="bill_depth_mm", hue="species", ax=axes[2]); 
sns.kdeplot(data=penguins, x="body_mass_g", hue="species", ax=axes[3]); 
plt.subplots_adjust(wspace=0.5)

# **Prep your data for clustering**
- Select the predictors (attributes) you want to use 
- Normalise the data (assuming it is normally distributed), otherwise the largest field will dominate (because we are going to minimise the Euclidean distance)

**Select the attributes** 
- I've chosen three: 

`attributes = ['bill_length_mm','bill_depth_mm','flipper_length_mm']`
- Have a play with different attributes, but note that you will have to also adjust some of the plots/evaluation metrics later on if you do

In [ ]:
attributes = ['bill_length_mm','bill_depth_mm','flipper_length_mm']

**Normalise the data**
- Check: 
  - i: number of observations
  - j: number of attributes 

- Check: 
  - Normalisation worked


In [ ]:
data = penguins[attributes].dropna().copy() # select the attributes from the original dataframe & make sure no nans! 
X = data.to_numpy() # convert them into a numpy array 
X.shape # check what shape X is 

In [ ]:
XNorm = preprocessing.normalize(X,axis=0,) # normalise each feature
XNorm.min(axis=0), XNorm.max(axis=0) # Check data, eg. see that it is between 0 and 1

# **Choosing the number of clusters**

Regardless of clustering method, we need to pre-assign the number of clusters *n*. 

By using the penguins dataset, we already know what we are trying to predict and so we have an idea of how many clusters to choose (i.e we know the structure of the data). In many instances, we don't know this already, and so have to estimate how many clusters to use. In these cases, there is no right or wrong answer. There are a few guidance metrics, but it ultimately comes down to you and your knowledge of the data. 

- I've chosen to predict species - we know there are 3 groups

`n = 3 `

- Change if you're predicting something else (eg. sex), or just want to see what happens! 

In [ ]:
n = 3 # number of clusters 

**Metrics for deciding cluster number**

There are a lot of metrics to choose from to help inform your cluster number choice. Here are three:
- Calinski Harabasz Score: The larger the score the more dense/separated the clusters 
- Silhouette Score: The higher the score the more well defined the clusters are
- Davies-Bouldin Index: Values closer to zero indicate less similarity between clusters 

Read more about these and others here: https://scikit-learn.org/stable/modules/clustering#clustering-performance-evaluation


In [ ]:
#@markdown Calculate the metrics
for k in range(2,6):
  kmeanstest = KMeans(n_clusters=k, random_state=0).fit(XNorm)
  print('n =',k) 
  print('Calinski Harabasz Score = ', metrics.calinski_harabasz_score(XNorm, kmeanstest.labels_))
  print('Silhouette Score = ', metrics.silhouette_score(XNorm, kmeanstest.labels_, metric='euclidean'))
  print('Davies-Bouldin Index = ', metrics.davies_bouldin_score(XNorm, kmeanstest.labels_))
  print('  ')



We can see very clearly in this case that three clusters is the optimal number. Be warned however, often these metrics give different answers and are entirely unhelpful (for example see Section 2.5 in https://www.publish.csiro.au/es/pdf/ES20003), hence you often have to decide for yourself. 

#**K-means clustering**

K-means aims to minimise the within-cluster sum-of-squares. K-means partitions the observations into *n* clusters using an iterative approach, each time comparing the cluster (or centroid) mean to the previous iteration until little change is found. The result is that each observation is assigned to the cluster with the nearest mean. Read more here: https://scikit-learn.org/stable/modules/clustering.html#k-means

In this section we:  
- Run through K-means 
- Look at some of the outputs 
- Plot 

**Run K-means** 
- Very easy on a small data set!

In [ ]:
kmeans = KMeans(n_clusters=n, random_state=0).fit(XNorm) # Perform clustering 

**Explore some of the outputs**
- `cluster_centres_` are the centroid means 
- `labels_` are the predicted labels [0,1,..,n]

In [ ]:
kmeans.cluster_centers_ 

In [ ]:
kmeans.labels_ 

In [ ]:
#@markdown **Plot 2D scatter plots**
#@markdown - If you are using different predictors to my example, you may have to alter these plots

# Add the labels into the dataframe holding the predictors 
data['k-means species']=kmeans.labels_ 

# Plot
fig, axes = plt.subplots(1,2,figsize=(10,4))
sns.scatterplot(data=data,x='flipper_length_mm',y='bill_length_mm',hue='k-means species',palette='flare',ax = axes[0]);
axes[0].set_title('Predicted')

sns.scatterplot(data=penguins,x='flipper_length_mm',y='bill_length_mm',hue='species',palette='flare',ax = axes[1]);
axes[1].set_title('Observed');

In [ ]:
#@markdown **Make a 3D scatter plot** { display-mode: "both" }
#@markdown - Again - if you have chosen different predictors, you may need to alter
fig = plt.figure(figsize=(8,5))
ax = plt.axes(projection='3d')
p = ax.scatter3D(data['flipper_length_mm'], 
                 data['bill_length_mm'], 
                 data['bill_depth_mm'], 
                 c=data['k-means species'], 
                 cmap=sns.color_palette('flare',as_cmap=True)); 
cbar = plt.colorbar(p,drawedges=True,ticks=[0,1,2],boundaries=np.arange(-0.5,3.5,1)); 
ax.view_init(20, 10) 
ax.set_xlabel('Flipper Length (mm)');
ax.set_ylabel('Bill Length (mm)');
ax.set_zlabel('Bill Depth (mm)');
plt.title('Predicted penguin data');


# **Evaluate**

To simply evaluate how well the k-means model has performed we can look at: 
- The means of the observed vs predicted clusters
- Plot a confusion matrix


*For each of these metrics, we need to make sure the labels refer to the same species!*

***Note** These evaluation techniques are only appropriate if you know the right anwser. In many applications of clustering, you will not, and you will have to rely on your knowlege of the data (i.e. its physical meaning) to understand if the clustering makes sense (eg. as in https://www.publish.csiro.au/es/pdf/ES20003) 

**Means** 
- Using the Pandas DataFrames makes this really easy
- If you have changed the clustering method - you will need to check that the species names are assigned correctly



In [ ]:
#@markdown Calculate means & differences

mns = data.groupby('k-means species').mean()[attributes]  # need to assign the correct species to the predicted labels
mns.index = ['Adelie','Gentoo','Chinstrap']             # if you have changed the clustering method, you will need to adjust this
print('Predicted')
print(mns)                                              # print the predicted means
print(' ')
print('Observed')
print(penguins.groupby('species').mean()[attributes])     # print the observed means for each species 
print(' ')
print('Differences')
print(penguins.groupby('species').mean()[attributes]-mns) # print the differences

**Confusion matrix**
- A confusion matrix shows the number of correctly and incorrectly assigned labels, for each different category. 
- A perfectly predicted data set would result in a diagonal line!
- We need to make sure that the 0,1,2s of the predicted model refer to the same as that for the observed. If you've altered the attributes/cluster numbers - this may need to be changed! 

In [ ]:
#@markdown Plot confusion matrix
# Get the observed species 
attributes2 = attributes.copy()
attributes2.append('species')
obs = penguins[attributes2].dropna().copy()['species']

# And the predicted species - again, if the you've changed the code - check! 
pred = data['k-means species'].copy()
pred[pred==0] = 'Adelie'
pred[pred==1] = 'Gentoo'
pred[pred==2] = 'Chinstrap'

# make the confustion matrix
cm = metrics.confusion_matrix(y_true=obs, y_pred=pred)

# Plot it
fig, axes = plt.subplots(1,1,figsize=(10,4))
plt.imshow(cm,interpolation='none',cmap='Blues')
for (i, j), z in np.ndenumerate(cm):
    plt.text(j, i, z, ha='center', va='center')
plt.xlabel("K-means")
plt.ylabel("Observed")
plt.xticks([0,1,2])
plt.yticks([0,1,2])
axes.set_xticklabels(['A','G','C'])
axes.set_yticklabels(['A','G','C']);

# **Hierarchical - Agglomerative Clustering** (in brief)

We can re-do the clusters using a range of different agglomerative clustering methods. Agglomerative clustering is a bottom up approach, where each observations is it's own cluster to start with and at each iteration may be merged with another, until there is only one cluster. 

This can be of benefit for data in which there is observations-connectivity, eg. spatial data. You can see an example of agglomerative clustering in this paper: https://www.publish.csiro.au/es/pdf/ES20003

There are a number of agglomerative clustering methods, which use different 'linkages', i.e. what they try to minimize to form the clusters. You can read about them here: https://scikit-learn.org/stable/modules/clustering#hierarchical-clustering

In this notebook, we will just look at the Ward linkage method, which is very similar to that of k-means in that it minimizes the within cluster sum of squares.

Here we will: 
- Look at the structure of the data to help understand how many clusters we should choose
- Compare the to k-means
- I've chosen to look at Ward linkage, but you can test the other methods! 

```
link = 'ward'
```




In [ ]:
link = 'ward' # or try: 'average', 'single' or 'complete'

In [ ]:
#@markdown **Dendrograms** { display-mode: "both" }
#@markdown - Dendrograms tell us about the structure of the data and how the clusters are merged for the given method 
#@markdown - Where there are large 'distances' (along y-axis) between merges of clusters implies less similarity between clusters

# Function
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    ''' This was taken from:
    https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html
    '''

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


# create model 
model = AgglomerativeClustering(linkage=link,n_clusters=None,distance_threshold=0).fit(XNorm)

# plot
plot_dendrogram(model)
plt.title('{} linkage dendrogram'.format(link));

In [ ]:
#@markdown **Agglomerative clustering**
#@markdown - Compare to k-means and obs

fig, axes = plt.subplots(1,3,figsize=(14,4))

# run the agglomerate model
agglom = AgglomerativeClustering(linkage=link,n_clusters=n).fit(XNorm)
data['{} species'.format(link)] = agglom.labels_

# plot
sns.scatterplot(x=X[:,1],y=X[:,0],hue=data['{} species'.format(link)],
                palette='flare',ax = axes[0]);
axes[0].set_title(link)

sns.scatterplot(x=X[:,1],y=X[:,0],hue=data['k-means species'],
                palette='flare',ax = axes[1]);
axes[1].set_title('k-means')

sns.scatterplot(data=penguins,x='bill_depth_mm',y='bill_length_mm',hue='species',
                palette='flare',ax = axes[2]);
axes[2].set_title('observed');

#Acknowledgments: 

- [Scikit-learn: Machine Learning in Python](http://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html), Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
- [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/)
- Hastie, T., Friedman, J., and Tibshirani, R. (2009). [The Elements of Statistical Learning]( https://web.stanford.edu/~hastie/ElemStatLearn/). Springer Series in Statistics. (Springer: New York, NY.)
- Kate Saunders (QUT - statistics master)

